# AI-PSCI-012: ESMFold for Rapid Structure Prediction

**AI in Pharmaceutical Sciences: Bench to Bedside**  
VCU School of Pharmacy | VIP Program | Spring 2026

---

**Week 6 | Module: AI Tools | Estimated Time: 60-90 minutes**

**Prerequisites**: AI-PSCI-001 through AI-PSCI-011

**GPU Recommended**: ESMFold runs faster with GPU but works on CPU too.

---

## 🎯 Learning Objectives

After completing this talktorial, you will be able to:

1. Run ESMFold for fast protein structure prediction
2. Compare ESMFold predictions with AlphaFold2 results
3. Understand the speed/accuracy tradeoffs between methods
4. Process multiple protein variants efficiently in batch
5. Interpret pLDDT confidence scores from ESMFold

---

## 📚 Background

### ESMFold: Speed Meets Accuracy

**ESMFold** is Meta AI's structure prediction model released in 2022. Unlike AlphaFold2, which requires a Multiple Sequence Alignment (MSA) search taking 5-15 minutes, ESMFold predicts structures from **single sequences** using a protein language model.

### Key Differences: ESMFold vs AlphaFold2

| Feature | ESMFold | AlphaFold2/ColabFold |
|---------|---------|----------------------|
| **Input** | Single sequence | Sequence + MSA |
| **Speed** | ~2 min/protein | ~15-30 min/protein |
| **Accuracy** | Very good | Excellent |
| **MSA Required** | No | Yes |
| **Best for** | Rapid screening, variants | Final structures, publication |
| **Confidence** | pLDDT | pLDDT + PAE |

### When to Use Each Method

- **ESMFold**: Rapid screening of many variants, quick structural checks, when MSA generation fails
- **AlphaFold2**: High-accuracy final structures, novel proteins, publication-quality models

### The ESM Model Family

ESMFold is built on the **ESM-2** protein language model, trained on 250 million protein sequences. The model learns:
- Evolutionary patterns from raw sequences
- Structural constraints implicitly
- No explicit evolutionary information (MSA) needed

### Key Concepts

- **pLDDT (predicted Local Distance Difference Test)**: Confidence score 0-100 per residue
  - >90: Very high confidence (typically accurate)
  - 70-90: Confident (backbone usually correct)
  - 50-70: Low confidence (may be disordered)
  - <50: Very low confidence (likely disordered)

- **Single-Sequence Prediction**: No need to search databases for homologs

- **Batch Processing**: Predict multiple sequences in one run

---

## 🛠️ Setup

Run this cell to install required packages. **This takes 1-2 minutes.**

In [ ]:
#@title 🛠️ Install Packages (1-2 min)
import sys

# Check for GPU
try:
    gpu_info = !nvidia-smi --query-gpu=name --format=csv,noheader
    if gpu_info and gpu_info[0] and 'NVIDIA' in gpu_info[0]:
        print(f"✅ GPU detected: {gpu_info[0]}")
        print("   ESMFold will run faster with GPU acceleration.")
    else:
        print("⚠️ No GPU detected. ESMFold will run on CPU (slower but works).")
except Exception:
    print("⚠️ Could not detect GPU. ESMFold will run on CPU.")

# Install required packages
print("\n📦 Installing ESMFold dependencies...")
!pip install transformers -q
!pip install accelerate -q
!pip install biopython py3Dmol matplotlib seaborn -q

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("\n✅ All packages installed!")

Import required libraries:

In [ ]:
#@title 📦 Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Bio.PDB import PDBParser, PDBIO, Superimposer
from Bio.PDB.Polypeptide import is_aa
import py3Dmol
import requests
import json
import os
import time
from pathlib import Path
from io import StringIO

# Set display options
pd.set_option('display.max_columns', 10)
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['figure.dpi'] = 100

print("✅ All libraries imported!")

---

## 🎯 Target Configuration

Select your drug target. This selection determines which protein you'll predict.

In [ ]:
#@title 🎯 Select Your Drug Target
TARGET = "DHFR" #@param ["DHFR", "ABL1", "EGFR", "AChE", "COX-2", "DPP-4"]

TARGET_CONFIG = {
    "DHFR": {
        "pdb": "1RX1", "uniprot": "P0ABQ4", "drug": "Trimethoprim", "organism": "E. coli",
        "mutations": ["L28R", "P21L", "A26T", "W30R", "I94L"],
        "mutation_context": "Resistance mutations reducing trimethoprim binding",
        "expected_length": 159
    },
    "ABL1": {
        "pdb": "1IEP", "uniprot": "P00519", "drug": "Imatinib", "organism": "Human",
        "mutations": ["T315I", "E255K", "Y253H", "G250E", "M351T"],
        "mutation_context": "Resistance mutations causing imatinib resistance in CML",
        "expected_length": 1130, "domain_start": 242, "domain_end": 493
    },
    "EGFR": {
        "pdb": "1M17", "uniprot": "P00533", "drug": "Erlotinib", "organism": "Human",
        "mutations": ["T790M", "L858R", "C797S", "G719S", "L861Q"],
        "mutation_context": "Drug resistance and activating mutations in lung cancer",
        "expected_length": 1210, "domain_start": 712, "domain_end": 979
    },
    "AChE": {
        "pdb": "4EY7", "uniprot": "P22303", "drug": "Donepezil", "organism": "Human",
        "mutations": ["Y337A", "F338A", "W286A", "E202Q", "S203A"],
        "mutation_context": "Mutations in catalytic gorge affecting substrate binding",
        "expected_length": 614
    },
    "COX-2": {
        "pdb": "3LN1", "uniprot": "P35354", "drug": "Celecoxib", "organism": "Human",
        "mutations": ["V523I", "S530A", "Y385F", "R120A", "H90Q"],
        "mutation_context": "Mutations affecting selectivity and catalysis",
        "expected_length": 604
    },
    "DPP-4": {
        "pdb": "1X70", "uniprot": "P27487", "drug": "Sitagliptin", "organism": "Human",
        "mutations": ["S630A", "E205A", "E206A", "Y547A", "R125A"],
        "mutation_context": "Catalytic and binding site mutations",
        "expected_length": 766
    }
}

config = TARGET_CONFIG[TARGET]
print(f"✅ Target: {TARGET}")
print(f"   PDB: {config['pdb']} | UniProt: {config['uniprot']}")
print(f"   Reference Drug: {config['drug']}")
print(f"   Organism: {config['organism']}")
print(f"\n🧬 Mutations to analyze: {', '.join(config['mutations'])}")
print(f"   Context: {config['mutation_context']}")

# Create output directory
output_dir = Path(f"esmfold_{TARGET}")
output_dir.mkdir(exist_ok=True)
print(f"\n📁 Output directory: {output_dir}")

---

## 🔬 Guided Inquiry 1: Fetching Protein Sequence

### Context

Before running ESMFold, we need the protein sequence from UniProt. For large proteins (kinases), we'll use just the catalytic domain for faster prediction.

### Your Task

Using your AI assistant, write code to:

1. Fetch your target's protein sequence from UniProt
2. For kinases (ABL1, EGFR), extract just the kinase domain
3. Display sequence information (length, first/last residues)

💡 **Prompting Tips**:
- Ask: "How do I fetch a protein sequence from UniProt REST API?"
- Ask: "How do I extract a subsequence in Python?"
- The domain boundaries are stored in `config['domain_start']` and `config['domain_end']`

### Verification

After running your code, confirm:
- [ ] Sequence is fetched successfully
- [ ] Sequence length matches expected (full or domain)
- [ ] Sequence is stored for later use

📓 **Lab Notebook**: Record your sequence length. For kinases, note which domain you're using and why this speeds up prediction.

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 2: Running ESMFold via API

### Context

ESMFold can be run through Meta's public API, which is the easiest approach. The API accepts a sequence and returns a PDB structure with pLDDT scores.

### Your Task

Using your AI assistant, write code to:

1. Submit the wild-type sequence to ESMFold API
2. Parse the returned PDB structure
3. Extract pLDDT confidence scores
4. Save the predicted structure

💡 **Prompting Tips**:
- Ask: "How do I use the ESMFold API from Meta?"
- The API endpoint is: `https://api.esmatlas.com/foldSequence/v1/pdb/`
- pLDDT scores are stored in the B-factor column of PDB files
- **Important**: The ESM Atlas API returns pLDDT in 0-1 range. Multiply by 100 to get standard 0-100 scale.

### Verification

After running your code, confirm:
- [ ] Structure is predicted successfully
- [ ] PDB file is saved
- [ ] Mean pLDDT > 70 (indicates good quality prediction)
- [ ] Prediction time is ~1-2 minutes

📓 **Lab Notebook**: Record the prediction time and mean pLDDT score.

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 3: Visualizing the Predicted Structure

### Context

Let's visualize our ESMFold prediction colored by pLDDT confidence. This helps identify well-predicted regions vs uncertain areas.

### Your Task

Using your AI assistant, write code to:

1. Load the predicted PDB structure
2. Create a 3D visualization colored by pLDDT (B-factor)
3. Create a pLDDT plot vs residue number
4. Identify regions of high/low confidence

💡 **Prompting Tips**:
- Ask: "How do I color a protein by B-factor in py3Dmol?"
- The color scheme should go from red (low) to blue (high)
- Use `spectrum` coloring with `prop='b'` for B-factor

### Verification

After running your code, confirm:
- [ ] 3D structure is displayed
- [ ] Colors match pLDDT values (blue = high confidence)
- [ ] pLDDT plot shows confidence across the sequence

📓 **Lab Notebook**: Identify which regions have low confidence. Are these loops, termini, or other features?

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 4: Batch Prediction of Mutant Variants

### Context

ESMFold's speed advantage really shines when predicting multiple variants. Let's predict structures for all clinically relevant mutations in our target.

### Your Task

Using your AI assistant, write code to:

1. Create mutant sequences for each mutation in `config['mutations']`
2. Predict structures for all mutants using ESMFold
3. Save each predicted structure
4. Collect pLDDT scores for comparison

💡 **Prompting Tips**:
- Ask: "How do I introduce a point mutation into a protein sequence?"
- Parse mutation notation like "L28R" (Leucine at position 28 → Arginine)
- Remember to account for domain offset if using a subdomain

### Verification

After running your code, confirm:
- [ ] All mutations are correctly introduced
- [ ] All structures are predicted successfully
- [ ] PDB files are saved for each variant
- [ ] Total time is reasonable (~5-10 minutes for 5 variants)

📓 **Lab Notebook**: Record prediction times for each variant. Note any that fail or have unusually low confidence.

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 5: Comparing pLDDT Across Variants

### Context

Now let's compare how mutations affect prediction confidence. Significant drops in pLDDT at the mutation site might indicate structural destabilization.

### Your Task

Using your AI assistant, write code to:

1. Create a summary table of all variants with mean pLDDT
2. Plot pLDDT comparison across variants
3. Compare pLDDT specifically at mutation sites
4. Identify any variants with notably lower confidence

💡 **Prompting Tips**:
- Ask: "How do I create a grouped bar chart comparing multiple variants?"
- Consider plotting pLDDT difference from wild-type
- Highlight mutation positions in plots

### Verification

After running your code, confirm:
- [ ] Summary table shows all variants
- [ ] pLDDT comparison plot is informative
- [ ] Any significant differences are highlighted

📓 **Lab Notebook**: Which mutations show the largest changes in pLDDT? What might this indicate about structural effects?

In [ ]:
# Your code here


---

## 🔬 Guided Inquiry 6: Speed Comparison with AlphaFold2

### Context

One of ESMFold's key advantages is speed. Let's quantify this advantage and discuss when to use each method.

### Your Task

Using your AI assistant, write code to:

1. Calculate total ESMFold prediction time for all variants
2. Estimate equivalent AlphaFold2 time (15-20 min per protein)
3. Create a visualization comparing the approaches
4. Write recommendations for when to use each method

💡 **Prompting Tips**:
- Ask: "How do I create a comparison chart showing time differences?"
- Consider a table format for the summary
- Think about use cases: rapid screening vs final structures

### Verification

After running your code, confirm:
- [ ] Time comparison is clear
- [ ] Recommendations are practical
- [ ] Tradeoffs are discussed

📓 **Lab Notebook**: Based on your experience, when would you use ESMFold vs AlphaFold2 in a drug discovery project?

In [ ]:
# Your code here


---

## ✅ Checkpoint

Before moving on to the next talktorial, confirm you can:

- [ ] Run ESMFold predictions via the API
- [ ] Extract and interpret pLDDT confidence scores
- [ ] Process multiple protein variants in batch
- [ ] Compare ESMFold results across variants
- [ ] Explain when to use ESMFold vs AlphaFold2
- [ ] Identify speed/accuracy tradeoffs

### Your lab notebook should include:

- [ ] pLDDT scores for wild-type and all mutants
- [ ] Prediction time comparison
- [ ] Visualization of pLDDT changes
- [ ] Notes on which mutations show largest effects
- [ ] Recommendations for your research workflow

---

## 🤔 Reflection Questions

Answer these in your lab notebook:

1. **Method Selection**: Your PI asks you to predict structures for 500 single-point mutations. Which method would you recommend and why? How would your answer change if they only needed 5 structures?

2. **Confidence Interpretation**: One of your mutants has lower pLDDT than the wild-type. Does this mean the mutation destabilizes the protein? What additional experiments or analyses could help answer this?

3. **Pipeline Design**: How would you combine ESMFold and AlphaFold2 in a drug discovery pipeline? Sketch out a workflow for identifying resistance mutations in your target.

---

## 📖 Further Reading

- [ESMFold Paper (Lin et al., 2023)](https://www.science.org/doi/10.1126/science.ade2574) - Original Science paper
- [ESM Atlas](https://esmatlas.com/) - Metagenomic structure database
- [AlphaFold vs ESMFold Comparison](https://www.nature.com/articles/s41592-022-01488-1) - Performance benchmarks
- [HuggingFace ESMFold](https://huggingface.co/facebook/esmfold_v1) - Run ESMFold locally

---

## 🔗 Connection to Research

ESMFold enables rapid structural screening that was previously impractical:

- **Variant Effect Prediction**: Screen thousands of mutations for structural effects
- **Protein Design**: Quick feedback on designed sequences
- **Orphan Proteins**: Predict structures when no homologs exist (MSA fails)
- **Industrial Scale**: Predict structures for entire proteomes

In the next talktorial (AI-PSCI-013), you'll learn about **ESM-1v** — which uses the same ESM model family to predict mutation effects on protein function without needing structure prediction at all.

---

*AI-PSCI-012 Complete. Proceed to AI-PSCI-013: ESM-1v for Variant Effect Prediction.*